In [ ]:
import os
import requests
import PyPDF2
from langchain.agents import AgentType, initialize_agent
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import userdata

# Set up environment
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["API_NINJAS_KEY"] = userdata.get('API_NINJAS_KEY')

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Initialize embeddings for RAG and preferences
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=userdata.get('GOOGLE_API_KEY'))

# Process PDF for Onam-specific private recipes
def extract_text_from_pdf(pdf_path: str) -> str:
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error extracting PDF: {str(e)}"

# Initialize FAISS vector store
def create_vector_store(texts):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text("\n".join(texts))
    return FAISS.from_texts(chunks, embeddings)

# Sample Onam recipes from PDF (replace with actual PDF path)
pdf_path = "/content/onam-recipes.pdf"
private_recipes_text = extract_text_from_pdf(pdf_path) if os.path.exists(pdf_path) else "Error extracting PDF."

vector_store = create_vector_store([private_recipes_text])
retriever = vector_store.as_retriever(search_kwargs={"k": 2, "score_threshold": 0.7})

# Initialize FAISS for user preferences
preferences_store = create_vector_store(["No user preferences stored yet."])
preference_retriever = preferences_store.as_retriever()

# Tools
def recipe_search(query: str) -> str:
    """Search for non-Onam recipes using TheMealDB API."""
    if "onam" in query.lower():
        return "Error: Onam recipes are available only through PrivateOnamRecipeSearch."
    url = f"https://www.themealdb.com/api/json/v1/1/search.php?s={query}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data.get("meals"):
            meal = data["meals"][0]
            ingredients = [meal[f"strIngredient{i}"] for i in range(1, 21) if meal.get(f"strIngredient{i}") and meal[f"strIngredient{i}"].strip()]
            preferences = preference_retriever.get_relevant_documents("user preferences")
            preference_text = "\n".join([doc.page_content for doc in preferences]) if preferences else ""
            for ingredient in ingredients:
                if preference_text and ingredient.lower() in preference_text.lower():
                    return f"Warning: Recipe contains {ingredient}, which may conflict with preferences: {preference_text}"
            return f"Recipe: {meal['strMeal']}\nIngredients: {', '.join(ingredients)}\nInstructions: {meal['strInstructions']}"
        return "No recipes found."
    except requests.RequestException as e:
        return f"Error fetching recipe: {str(e)}"

def extract_ingredients(recipe: str) -> str:
    """Extract ingredients from a recipe string using LLM inference."""
    try:
        prompt = f"""
        Extract the ingredients from the following recipe text. If no explicit ingredient list is found, infer the ingredients based on the text and common culinary knowledge. Return a comma-separated list of ingredients or 'No ingredients found' if none can be identified.
        Recipe: {recipe}
        """
        result = llm.invoke(prompt).content.strip()
        return result if result and "No ingredients found" not in result else "No ingredients found."
    except Exception:
        return "Error extracting ingredients."

def nutrition_analysis(query: str) -> str:
    """Analyze nutritional content using API-Ninjas Nutrition API."""
    api_key = userdata.get('API_NINJAS_KEY')
    url = f"https://api.api-ninjas.com/v1/nutrition?query={query}"
    try:
        response = requests.get(url, headers={'X-Api-Key': api_key})
        response.raise_for_status()
        data = response.json()
        if data:
            result = ""
            for item in data:
                result += f"Food: {item['name']}\nCalories: {item['calories']} kcal\nFat: {item['fat_total_g']}g\nProtein: {item['protein_g']}g\nCarbs: {item['carbohydrates_total_g']}g\n"
            return result
        return "No nutrition data found."
    except requests.RequestException as e:
        return f"Error fetching nutrition data: {str(e)}"

def store_preference(preference: str) -> str:
    """Store user preference in the vector store."""
    preferences_store.add_texts([preference])
    return "Preference stored successfully."

def retrieve_preference(query: str) -> str:
    """Retrieve user preferences from the vector store."""
    results = preference_retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results]) if results else "No preferences found."

def store_onam_recipe(recipe: str) -> str:
    """Store an Onam-specific private recipe in the vector store."""
    if "onam" not in recipe.lower():
        return "Error: Only Onam-specific recipes can be stored."
    vector_store.add_texts([recipe])
    return "Onam recipe stored successfully."

def retrieve_onam_recipe(query: str) -> str:
    """Retrieve Onam-specific private recipes using RAG."""
    if "onam" not in query.lower():
        query = "Onam " + query
    results = retriever.get_relevant_documents(query)
    if results:
        combined_text = "\n".join([doc.page_content for doc in results])
        prompt = f"""
        Format the following text into a coherent Onam recipe with ingredients and instructions. If ingredients are missing, infer them based on the text and common Onam dishes (e.g., payasam, sambar, avial). Return in the format:
        Recipe: [Name]
        Ingredients: [Comma-separated list]
        Instructions: [Steps]
        If no recipe can be formed, return 'No Onam recipes found.'
        Text: {combined_text}
        """
        return llm.invoke(prompt).content.strip()
    return "No Onam recipes found."

# Define Tools
tools = [
    Tool(
        name="RecipeSearch",
        func=recipe_search,
        description="Search for non-Onam recipes using TheMealDB API."
    ),
    Tool(
        name="ExtractIngredients",
        func=extract_ingredients,
        description="Extract ingredients from a recipe string for further processing, using LLM inference for unstructured text."
    ),
    Tool(
        name="NutritionAnalysis",
        func=nutrition_analysis,
        description="Analyze nutritional content of ingredients or recipes using API-Ninjas. Feed in all ingredients as a single query"
    ),
    Tool(
        name="StorePreference",
        func=store_preference,
        description="Store user preferences (e.g., allergies) in the vector database."
    ),
    Tool(
        name="RetrievePreference",
        func=retrieve_preference,
        description="Retrieve user preferences from the vector database."
    ),
    Tool(
        name="PrivateOnamRecipeSearch",
        func=retrieve_onam_recipe,
        description="Search for Onam-specific private recipes using RAG."
    )
]

# Memory Setup
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize Single Agent
chef_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True
)

# Main Interaction Loop
def main():
    print("Welcome to the Chef Assistant!")
    print("Enter your query (e.g., 'Find Onam recipes and their nutritional value', 'Find chicken soup recipe and its nutritional value', 'I’m allergic to peanuts', 'Retrieve my family Onam payasam recipe') or 'exit' to quit.")
    while True:
        user_input = input("Please enter your input: ")
        if user_input.lower() == 'exit':
            break
        # Check preferences before processing
        preferences = preference_retriever.get_relevant_documents("user preferences")
        preference_text = "\n".join([doc.page_content for doc in preferences]) if preferences else ""
        if preference_text:
            user_input += f" (Note: Consider user preferences - {preference_text})"
        response = chef_agent.run(user_input)
        print(response)

if __name__ == "__main__":
    main()

Welcome to the Chef Assistant!
Enter your query (e.g., 'Find Onam recipes and their nutritional value', 'Find chicken soup recipe and its nutritional value', 'I’m allergic to peanuts', 'Retrieve my family Onam payasam recipe') or 'exit' to quit.
Please enter your input: I have an allergy to raisins. keep in mind


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: StorePreference
Action Input: Allergy: raisins
Observation: Preference stored successfully.
Thought:Do I need to use a tool? No
AI: I've noted that you have an allergy to raisins. I will keep this in mind when providing recipes or suggestions.

> Finished chain.
I've noted that you have an allergy to raisins. I will keep this in mind when providing recipes or suggestions.
Please enter your input: Give a recipie for Onam payasam and the nutritional info like carb and fats


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: PrivateOnamRecipeSearch
Action Input: 

In [ ]:
import os
import requests
import PyPDF2
from langchain.agents import AgentType, initialize_agent
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import userdata
import uuid

# Set up environment
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["API_NINJAS_KEY"] = userdata.get('API_NINJAS_KEY')

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Initialize embeddings for RAG and preferences
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=userdata.get('GOOGLE_API_KEY'))

# Process PDF for Onam-specific private recipes
def extract_text_from_pdf(pdf_path: str) -> str:
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error extracting PDF: {str(e)}"

# Initialize FAISS vector store
def create_vector_store(texts):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text("\n".join(texts))
    return FAISS.from_texts(chunks, embeddings)

# Sample Onam recipes from PDF (replace with actual PDF path)
pdf_path = "/content/onam-recipes.pdf"
private_recipes_text = extract_text_from_pdf(pdf_path) if os.path.exists(pdf_path) else "Error extracting PDF."

vector_store = create_vector_store([private_recipes_text])
retriever = vector_store.as_retriever(search_kwargs={"k": 2, "score_threshold": 0.7})

# Initialize FAISS for user preferences
preferences_store = create_vector_store(["No user preferences stored yet."])
preference_retriever = preferences_store.as_retriever()

# Tools
def recipe_search(query: str) -> str:
    """Search for non-Onam recipes using TheMealDB API."""
    if "onam" in query.lower():
        return "Error: Onam recipes are available only through PrivateOnamRecipeSearch."
    url = f"https://www.themealdb.com/api/json/v1/1/search.php?s={query}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data.get("meals"):
            results = []
            preferences = preference_retriever.get_relevant_documents("user preferences")
            preference_text = "\n".join([doc.page_content for doc in preferences]) if preferences else ""
            for meal in data["meals"][:3]:  # Limit to top 3 recipes
                ingredients = [meal[f"strIngredient{i}"] for i in range(1, 21) if meal.get(f"strIngredient{i}") and meal[f"strIngredient{i}"].strip()]
                for ingredient in ingredients:
                    if preference_text and ingredient.lower() in preference_text.lower():
                        return f"Warning: Recipe '{meal['strMeal']}' contains {ingredient}, which conflicts with preferences: {preference_text}. Try another recipe."
                results.append(f"Recipe: {meal['strMeal']}\nIngredients: {', '.join(ingredients)}\nInstructions: {meal['strInstructions']}\n")
            return "\n".join(results) if results else "No recipes found."
        return "No recipes found."
    except requests.RequestException as e:
        return f"Error fetching recipe: {str(e)}"

def extract_ingredients(recipe: str) -> str:
    """Extract ingredients from a recipe string using LLM inference."""
    try:
        prompt = f"""
        Extract the ingredients from the following recipe text. If no explicit ingredient list is found, infer the ingredients based on the text and common culinary knowledge. Return a comma-separated list of ingredients or 'No ingredients found' if none can be identified.
        Recipe: {recipe}
        """
        result = llm.invoke(prompt).content.strip()
        return result if result and "No ingredients found" not in result else "No ingredients found."
    except Exception:
        return "Error extracting ingredients."

def nutrition_analysis(recipe: str) -> str:
    """Analyze nutritional content using API-Ninjas Nutrition API with full recipe description."""
    ingredients = extract_ingredients(recipe)
    if "No ingredients found" in ingredients or "Error" in ingredients:
        return "Cannot analyze nutrition: No valid ingredients extracted."
    api_key = userdata.get('API_NINJAS_KEY')
    query = ingredients.replace(", ", " and ")  # Format for API
    url = f"https://api.api-ninjas.com/v1/nutrition?query={query}"
    try:
        response = requests.get(url, headers={'X-Api-Key': api_key})
        response.raise_for_status()
        data = response.json()
        if data:
            result = ""
            for item in data:
                result += f"Food: {item['name']}\nCalories: {item['calories']} kcal\nFat: {item['fat_total_g']}g\nProtein: {item['protein_g']}g\nCarbs: {item['carbohydrates_total_g']}g\n"
            return result
        return "No nutrition data found."
    except requests.RequestException as e:
        return f"Error fetching nutrition data: {str(e)}"

def store_preference(preference: str) -> str:
    """Store user preference in the vector store."""
    preferences_store.add_texts([preference])
    return "Preference stored successfully."

def retrieve_preference(query: str) -> str:
    """Retrieve user preferences from the vector store."""
    results = preference_retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results]) if results else "No preferences found."

def store_onam_recipe(recipe: str) -> str:
    """Store an Onam-specific private recipe in the vector store."""
    if "onam" not in recipe.lower():
        return "Error: Only Onam-specific recipes can be stored."
    vector_store.add_texts([recipe])
    return "Onam recipe stored successfully."

def retrieve_onam_recipe(query: str) -> str:
    """Retrieve Onam-specific private recipes using RAG."""
    if "onam" not in query.lower():
        query = "Onam " + query
    results = retriever.get_relevant_documents(query)
    if results:
        combined_text = "\n".join([doc.page_content for doc in results])
        prompt = f"""
        Format the following text into coherent Onam recipes with ingredients and instructions. If ingredients are missing, infer them based on the text and common Onam dishes (e.g., payasam, sambar, avial). Return in the format:
        Recipe: [Name]
        Ingredients: [Comma-separated list]
        Instructions: [Steps]
        If multiple recipes are identified, list them all, separated by '---'. If no recipe can be formed, return 'No Onam recipes found.'
        Text: {combined_text}
        """
        return llm.invoke(prompt).content.strip()
    return "No Onam recipes found."

# Define Tools
tools = [
    Tool(
        name="RecipeSearch",
        func=recipe_search,
        description="Search for non-Onam recipes using TheMealDB API. Returns up to 3 recipes."
    ),
    Tool(
        name="ExtractIngredients",
        func=extract_ingredients,
        description="Extract ingredients from a recipe string for further processing, using LLM inference for unstructured text."
    ),
    Tool(
        name="NutritionAnalysis",
        func=nutrition_analysis,
        description="Analyze nutritional content of a recipe by passing all ingredients as a single query to API-Ninjas."
    ),
    Tool(
        name="StorePreference",
        func=store_preference,
        description="Store user preferences (e.g., allergies) in the vector database."
    ),
    Tool(
        name="RetrievePreference",
        func=retrieve_preference,
        description="Retrieve user preferences from the vector database."
    ),
    Tool(
        name="PrivateOnamRecipeSearch",
        func=retrieve_onam_recipe,
        description="Search for Onam-specific private recipes using RAG."
    )
]

# System Prompt
system_prompt = """
You are a Chef Assistant designed to help users with culinary queries, focusing on recipes, nutritional analysis, and user preferences. Handle the following scenarios:
1. **Recipe Search**: For non-Onam recipes, use RecipeSearch to fetch up to 3 recipes from TheMealDB API. For Onam-specific recipes, use PrivateOnamRecipeSearch to retrieve from the private vector store.
2. **Nutritional Analysis**: Use NutritionAnalysis to analyze the nutritional content of a recipe by passing all ingredients as a single query. If no ingredients are provided, extract them using ExtractIngredients first.
3. **User Preferences**: Always check user preferences via RetrievePreference before suggesting recipes. If a recipe contains ingredients that conflict with preferences (e.g., allergies), skip it or suggest an alternative by rewriting the recipe to exclude the conflicting ingredient(s).
4. **Onam Recipes**: Store Onam-specific recipes using StoreOnamRecipe and retrieve them with PrivateOnamRecipeSearch. Ensure recipes are formatted with ingredients and instructions.
5. **Multiple Results**: When multiple recipes are found, format them clearly, separated by '---'.
6. **Error Handling**: If a tool fails or no results are found, provide a clear error message and suggest alternatives if possible. If a tool gives an error, try to rectify it.
7. There might be diverse scenarios and you have to handle them accordingly.

**Important**: If user preferences indicate allergies or dietary restrictions, avoid recipes with those ingredients. If a recipe contains a conflicting ingredient, rewrite it by substituting the ingredient (e.g., replace peanuts with almonds) or skip it and explain why. Always prioritize user safety and satisfaction.You dont ask follow up questions unless its highly essential. Otherwise you can rewrite using your knowledge
"""

# Memory Setup
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize Single Agent
chef_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    agent_kwargs={"system_prompt": system_prompt}
)

# Main Interaction Loop
def main():
    print("Welcome to the Chef Assistant!")
    print("Enter your query (e.g., 'Find Onam recipes and their nutritional value', 'Find chicken soup recipe and its nutritional value', 'I’m allergic to peanuts', 'Retrieve my family Onam payasam recipe') or 'exit' to quit.")
    while True:
        user_input = input("Please enter your input: ")
        if user_input.lower() == 'exit':
            break
        # Check preferences before processing
        preferences = preference_retriever.get_relevant_documents("user preferences")
        preference_text = "\n".join([doc.page_content for doc in preferences]) if preferences else ""
        if preference_text:
            user_input += f" (Note: Consider user preferences - {preference_text})"
        response = chef_agent.run(user_input)
        print(response)

if __name__ == "__main__":
    main()

Welcome to the Chef Assistant!
Enter your query (e.g., 'Find Onam recipes and their nutritional value', 'Find chicken soup recipe and its nutritional value', 'I’m allergic to peanuts', 'Retrieve my family Onam payasam recipe') or 'exit' to quit.
Please enter your input: I have allergy towards peanuts


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: StorePreference
Action Input: allergy towards peanuts
Observation: Preference stored successfully.
Thought:Do I need to use a tool? No
AI: I have noted your allergy to peanuts. I will keep this in mind when providing recipe suggestions.

> Finished chain.
I have noted your allergy to peanuts. I will keep this in mind when providing recipe suggestions.
Please enter your input: Find Onam payasam recipie and its nutritional info


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: PrivateOnamRecipeSearch
Action Input: Onam payasam
Observation: Recipe: Onam Payasam
Ingredients

In [ ]:
import os
import requests
import PyPDF2
from langchain.agents import AgentType, initialize_agent
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from google.colab import userdata
import uuid

# Set up environment
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["API_NINJAS_KEY"] = userdata.get('API_NINJAS_KEY')

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Initialize embeddings for RAG and preferences
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=userdata.get('GOOGLE_API_KEY'))

# Process PDF for Onam-specific private recipes
def extract_text_from_pdf(pdf_path: str) -> str:
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error extracting PDF: {str(e)}"

# Initialize FAISS vector store
def create_vector_store(texts):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text("\n".join(texts))
    return FAISS.from_texts(chunks, embeddings)

# Sample Onam recipes from PDF (replace with actual PDF path)
pdf_path = "/content/onam-recipes.pdf"
private_recipes_text = extract_text_from_pdf(pdf_path) if os.path.exists(pdf_path) else "Error extracting PDF."

# Initialize FAISS vector store with compression
vector_store = create_vector_store([private_recipes_text])
base_retriever = vector_store.as_retriever(search_kwargs={"k": 4, "score_threshold": 0.7})
compressor = LLMChainExtractor.from_llm(llm)
retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# Initialize FAISS for user preferences with compression
preferences_store = create_vector_store(["No user preferences stored yet."])
base_preference_retriever = preferences_store.as_retriever()
preference_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_preference_retriever
)

# Tools
def recipe_search(query: str) -> str:
    """Search for non-Onam recipes using TheMealDB API."""
    if "onam" in query.lower():
        return "Error: Onam recipes are available only through PrivateOnamRecipeSearch."
    url = f"https://www.themealdb.com/api/json/v1/1/search.php?s={query}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data.get("meals"):
            results = []
            preferences = preference_retriever.get_relevant_documents("user preferences")
            preference_text = "\n".join([doc.page_content for doc in preferences]) if preferences else ""
            for meal in data["meals"][:3]:  # Limit to top 3 recipes
                ingredients = [meal[f"strIngredient{i}"] for i in range(1, 21) if meal.get(f"strIngredient{i}") and meal[f"strIngredient{i}"].strip()]
                for ingredient in ingredients:
                    if preference_text and ingredient.lower() in preference_text.lower():
                        return f"Warning: Recipe '{meal['strMeal']}' contains {ingredient}, which conflicts with preferences: {preference_text}. Try another recipe."
                results.append(f"Recipe: {meal['strMeal']}\nIngredients: {', '.join(ingredients)}\nInstructions: {meal['strInstructions']}\n")
            return "\n".join(results) if results else "No recipes found."
        return "No recipes found."
    except requests.RequestException as e:
        return f"Error fetching recipe: {str(e)}"

def extract_ingredients(recipe: str) -> str:
    """Extract ingredients from a recipe string using LLM inference."""
    try:
        prompt = f"""
        Extract the ingredients from the following recipe text. If no explicit ingredient list is found, infer the ingredients based on the text and common culinary knowledge. Return a comma-separated list of ingredients or 'No ingredients found' if none can be identified.
        Recipe: {recipe}
        """
        result = llm.invoke(prompt).content.strip()
        return result if result and "No ingredients found" not in result else "No ingredients found."
    except Exception:
        return "Error extracting ingredients."

def nutrition_analysis(recipe: str) -> str:
    """Analyze nutritional content using API-Ninjas Nutrition API with full recipe description."""
    ingredients = extract_ingredients(recipe)
    if "No ingredients found" in ingredients or "Error" in ingredients:
        return "Cannot analyze nutrition: No valid ingredients extracted."
    api_key = userdata.get('API_NINJAS_KEY')
    query = ingredients.replace(", ", " and ")  # Format for API
    url = f"https://api.api-ninjas.com/v1/nutrition?query={query}"
    try:
        response = requests.get(url, headers={'X-Api-Key': api_key})
        response.raise_for_status()
        data = response.json()
        if data:
            result = ""
            for item in data:
                result += f"Food: {item['name']}\nCalories: {item['calories']} kcal\nFat: {item['fat_total_g']}g\nProtein: {item['protein_g']}g\nCarbs: {item['carbohydrates_total_g']}g\n"
            return result
        return "No nutrition data found."
    except requests.RequestException as e:
        return f"Error fetching nutrition data: {str(e)}"

def store_preference(preference: str) -> str:
    """Store user preference in the vector store."""
    preferences_store.add_texts([preference])
    return "Preference stored successfully."

def retrieve_preference(query: str) -> str:
    """Retrieve user preferences from the vector store with compression."""
    results = preference_retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results]) if results else "No preferences found."

def store_onam_recipe(recipe: str) -> str:
    """Store an Onam-specific private recipe in the vector store."""
    if "onam" not in recipe.lower():
        return "Error: Only Onam-specific recipes can be stored."
    vector_store.add_texts([recipe])
    return "Onam recipe stored successfully."

def retrieve_onam_recipe(query: str) -> str:
    """Retrieve Onam-specific private recipes using advanced RAG with compression."""
    if "onam" not in query.lower():
        query = "Onam " + query
    results = retriever.get_relevant_documents(query)
    if results:
        combined_text = "\n".join([doc.page_content for doc in results])
        prompt = f"""
        Format the following text into coherent Onam recipes with ingredients and instructions. If ingredients are missing, infer them based on the text and common Onam dishes (e.g., payasam, sambar, avial). Return in the format:
        Recipe: [Name]
        Ingredients: [Comma-separated list]
        Instructions: [Steps]
        If multiple recipes are identified, list them all, separated by '---'. If no recipe can be formed, return 'No Onam recipes found.'
        Text: {combined_text}
        """
        return llm.invoke(prompt).content.strip()
    return "No Onam recipes found."

# Define Tools
tools = [
    Tool(
        name="RecipeSearch",
        func=recipe_search,
        description="Search for non-Onam recipes using TheMealDB API. Returns up to 3 recipes."
    ),
    Tool(
        name="ExtractIngredients",
        func=extract_ingredients,
        description="Extract ingredients from a recipe string for further processing, using LLM inference for unstructured text."
    ),
    Tool(
        name="NutritionAnalysis",
        func=nutrition_analysis,
        description="Analyze nutritional content of a recipe by passing all ingredients as a single query to API-Ninjas."
    ),
    Tool(
        name="StorePreference",
        func=store_preference,
        description="Store user preferences (e.g., allergies) in the vector database."
    ),
    Tool(
        name="RetrievePreference",
        func=retrieve_preference,
        description="Retrieve user preferences from the vector database with compression for relevance."
    ),
    Tool(
        name="PrivateOnamRecipeSearch",
        func=retrieve_onam_recipe,
        description="Search for Onam-specific private recipes using advanced RAG with compression."
    )
]

# System Prompt
system_prompt = """
You are a Chef Assistant designed to help users with culinary queries, focusing on recipes, nutritional analysis, and user preferences. Handle the following scenarios:
1. **Recipe Search**: For non-Onam recipes, use RecipeSearch to fetch up to 3 recipes from TheMealDB API. For Onam-specific recipes, use PrivateOnamRecipeSearch to retrieve from the private vector store with advanced RAG compression.
2. **Nutritional Analysis**: Use NutritionAnalysis to analyze the nutritional content of a recipe by passing all ingredients as a single query. If no ingredients are provided, extract them using ExtractIngredients first. Try to summarize and give a concise nutritional analysis.
3. **User Preferences**: Always check user preferences via RetrievePreference before suggesting recipes. If a recipe contains ingredients that conflict with preferences (e.g., allergies), skip it or suggest an alternative by rewriting the recipe to exclude the conflicting ingredient(s).
4. **Onam Recipes**: Store Onam-specific recipes using StoreOnamRecipe and retrieve them with PrivateOnamRecipeSearch. Ensure recipes are formatted with ingredients and instructions.
5. **Multiple Results**: When multiple recipes are found, format them clearly, separated by '---'.
6. **Error Handling**: If a tool fails or no results are found, provide a clear error message and suggest alternatives if possible. If a tool gives an error, try to rectify it.
7. There might be diverse scenarios and you have to handle them accordingly.

**Important**: If user preferences indicate allergies or dietary restrictions, avoid recipes with those ingredients. If a recipe contains a conflicting ingredient, rewrite it by substituting the ingredient (e.g., replace peanuts with almonds) or skip it and explain why. Always prioritize user safety and satisfaction.
"""

# Memory Setup
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize Single Agent
chef_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    agent_kwargs={"system_prompt": system_prompt}
)

# Main Interaction Loop
def main():
    print("Welcome to the Chef Assistant!")
    print("Enter your query (e.g., 'Find Onam recipes and their nutritional value', 'Find chicken soup recipe and its nutritional value', 'I’m allergic to peanuts', 'Retrieve my family Onam payasam recipe') or 'exit' to quit.")
    while True:
        user_input = input("Please enter your input: ")
        if user_input.lower() == 'exit':
            break
        # Check preferences before processing
        preferences = preference_retriever.get_relevant_documents("user preferences")
        preference_text = "\n".join([doc.page_content for doc in preferences]) if preferences else ""
        if preference_text:
            user_input += f" (Note: Consider user preferences (if relevant to the query) - {preference_text})"
        response = chef_agent.run(user_input)
        print(response)

if __name__ == "__main__":
    main()

Welcome to the Chef Assistant!
Enter your query (e.g., 'Find Onam recipes and their nutritional value', 'Find chicken soup recipe and its nutritional value', 'I’m allergic to peanuts', 'Retrieve my family Onam payasam recipe') or 'exit' to quit.
Please enter your input: Find chicken soup recipe and fetch its nutritonal value


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: RecipeSearch
Action Input: chicken soup
Observation: Recipe: Rosol (Polish Chicken Soup)
Ingredients: Chicken Legs, Onions, Carrots, Leek, Celery, Cabbage, Cloves, Allspice, Bay Leaf, Parsley, Dill, Pepper, Salt
Instructions: Add chicken to a large Dutch oven or stock pot 
Cover with water
Bring to a boil and simmer for 2 to 2 1/2 hours, skimming any impurities off the top to insure a clear broth
If your pot is big enough, add the vegetables and spices for the last hour of the cooking time
My Dutch oven wasn’t big enough to hold everything, just the chicken and other bones filled 

KeyboardInterrupt: Interrupted by user

In [ ]:
import os
import requests
import PyPDF2
from langchain.agents import AgentType, initialize_agent
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from google.colab import userdata
import uuid

# Set up environment
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["API_NINJAS_KEY"] = userdata.get('API_NINJAS_KEY')

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Initialize embeddings for RAG and preferences
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=userdata.get('GOOGLE_API_KEY'))

# Process PDF for Onam-specific private recipes
def extract_text_from_pdf(pdf_path: str) -> str:
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error extracting PDF: {str(e)}"

# Initialize FAISS vector store
def create_vector_store(texts):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text("\n".join(texts))
    return FAISS.from_texts(chunks, embeddings)

# Sample Onam recipes from PDF (replace with actual PDF path)
pdf_path = "/content/onam-recipes.pdf"
private_recipes_text = extract_text_from_pdf(pdf_path) if os.path.exists(pdf_path) else "Error extracting PDF."

# Initialize FAISS vector store with compression
vector_store = create_vector_store([private_recipes_text])
base_retriever = vector_store.as_retriever(search_kwargs={"k": 4, "score_threshold": 0.7})
compressor = LLMChainExtractor.from_llm(llm)
retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# Initialize FAISS for user preferences with compression
preferences_store = create_vector_store(["No user preferences stored yet."])
base_preference_retriever = preferences_store.as_retriever()
preference_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_preference_retriever
)

# Tools
def recipe_search(query: str) -> str:
    """Search for non-Onam recipes using TheMealDB API."""
    if "onam" in query.lower():
        return "Error: Onam recipes are available only through PrivateOnamRecipeSearch."
    url = f"https://www.themealdb.com/api/json/v1/1/search.php?s={query}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data.get("meals"):
            results = []
            preferences = preference_retriever.get_relevant_documents("user preferences")
            preference_text = "\n".join([doc.page_content for doc in preferences]) if preferences else ""
            for meal in data["meals"][:3]:  # Limit to top 3 recipes
                ingredients = [meal[f"strIngredient{i}"] for i in range(1, 21) if meal.get(f"strIngredient{i}") and meal[f"strIngredient{i}"].strip()]
                for ingredient in ingredients:
                    if preference_text and ingredient.lower() in preference_text.lower():
                        return f"Warning: Recipe '{meal['strMeal']}' contains {ingredient}, which conflicts with preferences: {preference_text}. Try another recipe."
                results.append(f"Recipe: {meal['strMeal']}\nIngredients: {', '.join(ingredients)}\nInstructions: {meal['strInstructions']}\n")
            return "\n".join(results) if results else "No recipes found."
        return "No recipes found."
    except requests.RequestException as e:
        return f"Error fetching recipe: {str(e)}"

def extract_ingredients(recipe: str) -> str:
    """Extract ingredients from a recipe string using LLM inference."""
    try:
        prompt = f"""
        Extract the ingredients from the following recipe text. If no explicit ingredient list is found, infer the ingredients based on the text and common culinary knowledge. Return a comma-separated list of ingredients or 'No ingredients found' if none can be identified.
        Recipe: {recipe}
        """
        result = llm.invoke(prompt).content.strip()
        return result if result and "No ingredients found" not in result else "No ingredients found."
    except Exception:
        return "Error extracting ingredients."

def nutrition_analysis(recipe: str) -> str:
    """Analyze nutritional content using API-Ninjas Nutrition API with full recipe description."""
    ingredients = extract_ingredients(recipe)
    if "No ingredients found" in ingredients or "Error" in ingredients:
        return "Cannot analyze nutrition: No valid ingredients extracted."
    api_key = userdata.get('API_NINJAS_KEY')
    query = ingredients.replace(", ", " and ")  # Format for API
    url = f"https://api.api-ninjas.com/v1/nutrition?query={query}"
    try:
        response = requests.get(url, headers={'X-Api-Key': api_key})
        response.raise_for_status()
        data = response.json()
        if data:
            result = ""
            for item in data:
                result += f"Food: {item['name']}\nCalories: {item['calories']} kcal\nFat: {item['fat_total_g']}g\nProtein: {item['protein_g']}g\nCarbs: {item['carbohydrates_total_g']}g\n"
            return result
        return "No nutrition data found."
    except requests.RequestException as e:
        return f"Error fetching nutrition data: {str(e)}"

def store_preference(preference: str) -> str:
    """Store user preference in the vector store."""
    preferences_store.add_texts([preference])
    return "Preference stored successfully."

def retrieve_preference(query: str) -> str:
    """Retrieve user preferences from the vector store with compression."""
    results = preference_retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results]) if results else "No preferences found."

def store_onam_recipe(recipe: str) -> str:
    """Store an Onam-specific private recipe in the vector store."""
    if "onam" not in recipe.lower():
        return "Error: Only Onam-specific recipes can be stored."
    vector_store.add_texts([recipe])
    return "Onam recipe stored successfully."

def retrieve_onam_recipe(query: str) -> str:
    """Retrieve Onam-specific private recipes using advanced RAG with compression."""
    if "onam" not in query.lower():
        query = "Onam " + query
    results = retriever.get_relevant_documents(query)
    if results:
        combined_text = "\n".join([doc.page_content for doc in results])
        prompt = f"""
        Format the following text into coherent Onam recipes with ingredients and instructions. If ingredients are missing, infer them based on the text and common Onam dishes (e.g., payasam, sambar, avial). Return in the format:
        Recipe: [Name]
        Ingredients: [Comma-separated list]
        Instructions: [Steps]
        If multiple recipes are identified, list them all, separated by '---'. If no recipe can be formed, return 'No Onam recipes found.'
        Text: {combined_text}
        """
        return llm.invoke(prompt).content.strip()
    return "No Onam recipes found."

# Define Tools
tools = [
    Tool(
        name="RecipeSearch",
        func=recipe_search,
        description="Search for non-Onam recipes using TheMealDB API. Returns up to 3 recipes."
    ),
    Tool(
        name="ExtractIngredients",
        func=extract_ingredients,
        description="Extract ingredients from a recipe string for further processing, using LLM inference for unstructured text."
    ),
    Tool(
        name="NutritionAnalysis",
        func=nutrition_analysis,
        description="Analyze nutritional content of a recipe by passing all ingredients as a single query to API-Ninjas."
    ),
    Tool(
        name="StorePreference",
        func=store_preference,
        description="Store user preferences (e.g., allergies) in the vector database."
    ),
    Tool(
        name="RetrievePreference",
        func=retrieve_preference,
        description="Retrieve user preferences from the vector database with compression for relevance."
    ),
    Tool(
        name="PrivateOnamRecipeSearch",
        func=retrieve_onam_recipe,
        description="Search for Onam-specific private recipes using advanced RAG with compression."
    )
]

# System Prompt
system_prompt = """
You are a Chef Assistant designed to help users with culinary queries, focusing on recipes, nutritional analysis, and user preferences. Handle the following scenarios:
1. **Recipe Search**: For non-Onam recipes, use RecipeSearch to fetch up to 3 recipes from TheMealDB API. For Onam-specific recipes, use PrivateOnamRecipeSearch to retrieve from the private vector store with advanced RAG compression.
2. **Nutritional Analysis**: Use NutritionAnalysis to analyze the nutritional content of a recipe by passing all ingredients as a single query. If no ingredients are provided, extract them using ExtractIngredients first. Try to summarize and give a concise nutritional analysis.
3. **User Preferences**: Always check user preferences via RetrievePreference before suggesting recipes. If a recipe contains ingredients that conflict with preferences (e.g., allergies), skip it or suggest an alternative by rewriting the recipe to exclude the conflicting ingredient(s).
4. **Onam Recipes**: Store Onam-specific recipes using StoreOnamRecipe and retrieve them with PrivateOnamRecipeSearch. Ensure recipes are formatted with ingredients and instructions.
5. **Multiple Results**: When multiple recipes are found, format them clearly, separated by '---'.
6. **Error Handling**: If a tool fails or no results are found, provide a clear error message and suggest alternatives if possible. If a tool gives an error, try to rectify it.
7. There might be diverse scenarios and you have to handle them accordingly.
8. If a partcular recipie is not found, search for the base recipie and make the modifications yourself. Use the tools to accomplish parts of the query.

You may use nutritonal analysis tool to understand the nutrition of the recipie if the task directly or indirectly needs it.
**Important**: If user preferences indicate allergies or dietary restrictions, avoid recipes with those ingredients. If a recipe contains a conflicting ingredient, rewrite it by substituting the ingredient (e.g., replace peanuts with almonds) or skip it and explain why. Always prioritize user safety and satisfaction.
"""

# Memory Setup
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize Single Agent
chef_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    agent_kwargs={"system_prompt": system_prompt}
)

# Main Interaction Loop
def main():
    print("Welcome to the Chef Assistant!")
    print("Enter your query \n")
    while True:
        user_input = input("\nPlease enter your input: ")
        if user_input.lower() == 'exit':
            break
        # Check preferences before processing
        preferences = preference_retriever.get_relevant_documents("user preferences")
        preference_text = "\n".join([doc.page_content for doc in preferences]) if preferences else ""
        if preference_text:
            user_input += f" (Note: Consider user preferences (if relevant to the query) - {preference_text})"
        response = chef_agent.run(user_input)
        print(response)

if __name__ == "__main__":
    main()

Welcome to the Chef Assistant!
Enter your query 


Please enter your input: I have an allergy to raisins


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: StorePreference
Action Input: allergy to raisins
Observation: Preference stored successfully.
Thought:Do I need to use a tool? No
AI: I've noted that you have an allergy to raisins. I will keep this in mind for future recipe suggestions.

> Finished chain.
I've noted that you have an allergy to raisins. I will keep this in mind for future recipe suggestions.

Please enter your input: Find me Onam payasam recipie that are low in carb and fat


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: PrivateOnamRecipeSearch
Action Input: Onam payasam recipes low in carb and fat, excluding raisins

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 29
}
].



Observation: No Onam recipes found.
Thought:Do I need to use a tool? No
AI: I couldn't find any Onam payasam recipes that are specifically low in carb and fat. Would you like me to look for other types of Onam recipes, or perhaps explore payasam recipes without the low carb and fat restriction?

> Finished chain.
I couldn't find any Onam payasam recipes that are specifically low in carb and fat. Would you like me to look for other types of Onam recipes, or perhaps explore payasam recipes without the low carb and fat restriction?


KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.3 MB/s eta 0:00:00


In [ ]:
!pip install langchain langchain_community langchain-google-genai faiss-cpu pypdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires

In [ ]:
import os
import requests
import PyPDF2
from langchain.agents import AgentType, initialize_agent
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from google.colab import userdata
import uuid
from typing import List, Dict, Any

# Set up environment
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["API_NINJAS_KEY"] = userdata.get('API_NINJAS_KEY')

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

# Initialize embeddings for RAG and preferences
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=userdata.get('GOOGLE_API_KEY'))

# Process PDF for Onam-specific private recipes
def extract_text_from_pdf(pdf_path: str) -> str:
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error extracting PDF: {str(e)}"

# Initialize FAISS vector store
def create_vector_store(texts: List[str]) -> FAISS:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text("\n".join(texts))
    return FAISS.from_texts(chunks, embeddings)

# Sample Onam recipes from PDF (replace with actual PDF path)
pdf_path = "/content/onam-recipes.pdf"
private_recipes_text = extract_text_from_pdf(pdf_path) if os.path.exists(pdf_path) else "Error extracting PDF."

# Initialize FAISS vector store with compression
vector_store = create_vector_store([private_recipes_text])
base_retriever = vector_store.as_retriever(search_kwargs={"k": 4, "score_threshold": 0.7})
compressor = LLMChainExtractor.from_llm(llm)
retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# Initialize FAISS for user preferences with compression
preferences_store = create_vector_store(["No user preferences stored yet."])
base_preference_retriever = preferences_store.as_retriever()
preference_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_preference_retriever
)

# Enhanced Tools
def recipe_search(query: str) -> str:
    """Search for non-Onam recipes using TheMealDB API."""
    if "onam" in query.lower():
        return "Error: Onam recipes are available only through PrivateOnamRecipeSearch."
    url = f"https://www.themealdb.com/api/json/v1/1/search.php?s={query}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data.get("meals"):
            results = []
            for meal in data["meals"][:3]:  # Limit to top 3 recipes
                ingredients = [meal[f"strIngredient{i}"] for i in range(1, 21) if meal.get(f"strIngredient{i}") and meal[f"strIngredient{i}"].strip()]
                results.append({
                    "name": meal['strMeal'],
                    "ingredients": ingredients,
                    "instructions": meal['strInstructions']
                })
            return str(results)
        return "No recipes found."
    except requests.RequestException as e:
        return f"Error fetching recipe: {str(e)}"

def extract_ingredients(recipe: str) -> str:
    """Extract ingredients from a recipe string using LLM inference."""
    try:
        prompt = f"""
        Extract the ingredients from the following recipe text. Return a JSON object with:
        - "ingredients": list of ingredients
        - "possible_allergens": list of common allergens found (e.g., nuts, dairy)
        Recipe: {recipe}
        """
        result = llm.invoke(prompt).content.strip()
        return result
    except Exception as e:
        return f"Error extracting ingredients: {str(e)}"

def nutrition_analysis(recipe: str) -> str:
    """Analyze nutritional content using API-Ninjas Nutrition API."""
    try:
        # First extract ingredients
        ingredients_data = extract_ingredients(recipe)
        if "ingredients" in ingredients_data:
            ingredients = eval(ingredients_data)["ingredients"]
        else:
            # Fallback to simple extraction if JSON parsing fails
            prompt = f"Extract just the ingredients from: {recipe}"
            ingredients = llm.invoke(prompt).content.split(",")

        # Get nutrition data
        api_key = userdata.get('API_NINJAS_KEY')
        query = " and ".join(ingredients[:10])  # Limit to 10 ingredients for API
        url = f"https://api.api-ninjas.com/v1/nutrition?query={query}"
        response = requests.get(url, headers={'X-Api-Key': api_key})
        response.raise_for_status()
        data = response.json()

        # Calculate totals
        totals = {
            "calories": 0,
            "fat": 0,
            "protein": 0,
            "carbs": 0
        }
        for item in data:
            totals["calories"] += item['calories']
            totals["fat"] += item['fat_total_g']
            totals["protein"] += item['protein_g']
            totals["carbs"] += item['carbohydrates_total_g']

        return str({
            "per_serving": totals,
            "details": data
        })
    except Exception as e:
        return f"Error in nutrition analysis: {str(e)}"

def store_preference(preference: str) -> str:
    """Store user preference in the vector store."""
    preferences_store.add_texts([preference])
    return "Preference stored successfully."

def retrieve_preference(query: str = "") -> str:
    """Retrieve all user preferences from the vector store."""
    results = preference_retriever.get_relevant_documents("user preferences")
    return str([doc.page_content for doc in results]) if results else "No preferences found."

def store_onam_recipe(recipe: str) -> str:
    """Store an Onam-specific private recipe in the vector store."""
    if "onam" not in recipe.lower():
        return "Error: Only Onam-specific recipes can be stored."
    vector_store.add_texts([recipe])
    return "Onam recipe stored successfully."

def retrieve_onam_recipe(query: str) -> str:
    """Retrieve Onam-specific private recipes using advanced RAG."""
    if "onam" not in query.lower():
        query = "Onam " + query
    results = retriever.get_relevant_documents(query)
    return str([doc.page_content for doc in results]) if results else "No Onam recipes found."

def modify_recipe(recipe: str, modifications: str) -> str:
    """Modify a recipe based on requested changes."""
    try:
        prompt = f"""
        Modify the following recipe based on these requirements: {modifications}
        Keep the core essence of the dish while making the requested changes.
        Return the modified recipe in this format:
        Name: [recipe name]
        Ingredients: [list]
        Instructions: [steps]
        Modifications made: [summary]

        Original recipe: {recipe}
        """
        return llm.invoke(prompt).content
    except Exception as e:
        return f"Error modifying recipe: {str(e)}"

# Define Enhanced Tools
tools = [
    Tool(
        name="RecipeSearch",
        func=recipe_search,
        description="Search for non-Onam recipes from TheMealDB. Returns list of recipes with names, ingredients and instructions."
    ),
    Tool(
        name="ExtractIngredients",
        func=extract_ingredients,
        description="Extract ingredients and allergens from recipe text. Returns JSON with ingredients and possible allergens."
    ),
    Tool(
        name="NutritionAnalysis",
        func=nutrition_analysis,
        description="Analyze nutritional content of a recipe. Returns per-serving totals and detailed breakdown."
    ),
    Tool(
        name="StorePreference",
        func=store_preference,
        description="Store user dietary preferences/allergies in the vector database."
    ),
    Tool(
        name="RetrievePreference",
        func=retrieve_preference,
        description="Retrieve all stored user preferences from the vector database."
    ),
    Tool(
        name="PrivateOnamRecipeSearch",
        func=retrieve_onam_recipe,
        description="Search for Onam-specific private recipes from the vector store. Returns list of matching recipes."
    ),
    Tool(
        name="ModifyRecipe",
        func=modify_recipe,
        description="Modify a recipe to meet specific requirements (e.g., low carb, allergen-free). Returns adapted recipe."
    )
]

# Enhanced System Prompt
system_prompt = """
You are an Intelligent Chef Assistant with these capabilities:

1. **Query Understanding**:
   - Analyze the user's request to determine what they really need
   - Identify implicit requirements (e.g., "healthy" = low fat/sugar)
   - Consider context from conversation history

2. **Tool Selection Strategy**:
   - For Onam recipes: Use PrivateOnamRecipeSearch first
   - For other recipes: Use RecipeSearch
   - Always check preferences with RetrievePreference first
   - For nutritional needs: Use NutritionAnalysis
   - For modifications: Use ModifyRecipe

3. **Preference Handling**:
   - Before suggesting recipes, check for allergies/dietary restrictions
   - If conflicts exist, either:
     a) Find alternative recipes, or
     b) Modify recipes to accommodate preferences using ModifyRecipe
   - Clearly explain any modifications made

4. **Nutritional Requests**:
   - For "healthy", "low carb", etc. requests:
     a) Find recipes that naturally fit, or
     b) Modify existing recipes to meet criteria
     c) Provide nutritional analysis to verify

5. **Multi-Step Problem Solving**:
   - Combine tools when needed (e.g., find recipe -> check nutrition -> modify if needed)
   - Never give up after one tool fails - try alternative approaches

6. **Response Formatting**:
   - Present recipes clearly with name, ingredients, instructions
   - Highlight modifications made
   - Provide nutritional info when relevant
   - Explain your reasoning for suggestions

7. **Error Handling**:
   - If tools fail, try alternative approaches
   - Never say "I couldn't find" without trying all options
   - If no perfect match exists, provide the closest option with modifications

**Always** prioritize user preferences and dietary restrictions. Be proactive in finding solutions, not just reporting failures.
"""

# Memory Setup
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize Enhanced Agent
chef_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    agent_kwargs={
        "system_prompt": system_prompt,
        "prefix": "You are an expert chef assistant. Think step-by-step to provide the best culinary solutions."
    }
)

# Enhanced Main Interaction Loop
def main():
    print("Welcome to the Enhanced Chef Assistant!")
    print("Type 'exit' to end the session.\n")

    while True:
        try:
            user_input = input("\nYour culinary request: ").strip()
            if user_input.lower() in ['exit', 'quit']:
                break

            if not user_input:
                continue

            # Get current preferences for context
            prefs = retrieve_preference()
            if prefs != "No preferences found.":
                user_input += f" (Current preferences: {prefs})"

            # Process the request
            response = chef_agent.run({
                "input": user_input,
                "chat_history": memory.load_memory_variables({})['chat_history']
            })

            # Format the response
            print("\nChef Assistant:")
            print(response)

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("Let's try that again. Please rephrase your request.")

if __name__ == "__main__":
    main()

Welcome to the Enhanced Chef Assistant!
Type 'exit' to end the session.


Your culinary request: I am allergic to raisins


  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/

KeyboardInterrupt: 

In [ ]:
pip install openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.9 MB/s eta 0:00:00


In [ ]:
import os
import requests
import PyPDF2
from langchain.agents import AgentType, initialize_agent
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import userdata
import uuid

# Set up environment
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["API_NINJAS_KEY"] = userdata.get('API_NINJAS_KEY')

# Initialize LLM - Replaced with GPT-4o
#llm = ChatOpenAI(
#    model="gpt-4o",
#    temperature=0,
#    openai_api_key=os.environ["OPENAI_API_KEY"]
#)

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["API_NINJAS_KEY"] = userdata.get('API_NINJAS_KEY')

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Initialize embeddings (OpenAI)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=userdata.get('GOOGLE_API_KEY'))

# Process PDF for Onam-specific private recipes
def extract_text_from_pdf(pdf_path: str) -> str:
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error extracting PDF: {str(e)}"

# Initialize FAISS vector store
def create_vector_store(texts):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text("\n".join(texts))
    return FAISS.from_texts(chunks, embeddings)

# Sample Onam recipes from PDF (replace with actual PDF path)
pdf_path = "/content/onam-recipes.pdf"
private_recipes_text = extract_text_from_pdf(pdf_path) if os.path.exists(pdf_path) else "Error extracting PDF."

vector_store = create_vector_store([private_recipes_text])
retriever = vector_store.as_retriever()

# Initialize FAISS for user preferences
preferences_store = create_vector_store(["No user preferences stored yet."])
preference_retriever = preferences_store.as_retriever()

# Tools
def recipe_search(query: str) -> str:
    if "onam" in query.lower():
        return "Error: Onam recipes are available only through PrivateOnamRecipeSearch."
    url = f"https://www.themealdb.com/api/json/v1/1/search.php?s={query}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data.get("meals"):
            results = []
            preferences = preference_retriever.get_relevant_documents("user preferences")
            preference_text = "\n".join([doc.page_content for doc in preferences]) if preferences else ""
            for meal in data["meals"][:3]:
                ingredients = [meal[f"strIngredient{i}"] for i in range(1, 21) if meal.get(f"strIngredient{i}") and meal[f"strIngredient{i}"].strip()]
                for ingredient in ingredients:
                    if preference_text and ingredient.lower() in preference_text.lower():
                        return f"Warning: Recipe '{meal['strMeal']}' contains {ingredient}, which conflicts with preferences: {preference_text}. Try another recipe."
                results.append(f"Recipe: {meal['strMeal']}\nIngredients: {', '.join(ingredients)}\nInstructions: {meal['strInstructions']}\n")
            return "\n".join(results) if results else "No recipes found."
        return "No recipes found."
    except requests.RequestException as e:
        return f"Error fetching recipe: {str(e)}"

def extract_ingredients(recipe: str) -> str:
    try:
        prompt = f"""
        Extract the ingredients from the following recipe text. If no explicit ingredient list is found, infer them based on the text and common culinary knowledge. Return a comma-separated list of ingredients or 'No ingredients found' if none can be identified.
        Recipe: {recipe}
        """
        result = llm.invoke(prompt).content.strip()
        return result if result and "No ingredients found" not in result else "No ingredients found."
    except Exception:
        return "Error extracting ingredients."

def nutrition_analysis(recipe: str) -> str:
    ingredients = extract_ingredients(recipe)
    if "No ingredients found" in ingredients or "Error" in ingredients:
        return "Cannot analyze nutrition: No valid ingredients extracted."
    api_key = userdata.get('API_NINJAS_KEY')
    query = ingredients.replace(", ", " and ")
    url = f"https://api.api-ninjas.com/v1/nutrition?query={query}"
    try:
        response = requests.get(url, headers={'X-Api-Key': api_key})
        response.raise_for_status()
        data = response.json()
        if data:
            result = ""
            for item in data:
                result += f"Food: {item['name']}\nCalories: {item['calories']} kcal\nFat: {item['fat_total_g']}g\nProtein: {item['protein_g']}g\nCarbs: {item['carbohydrates_total_g']}g\n"
            return result
        return "No nutrition data found."
    except requests.RequestException as e:
        return f"Error fetching nutrition data: {str(e)}"

def store_preference(preference: str) -> str:
    preferences_store.add_texts([preference])
    return "Preference stored successfully."

def retrieve_preference(query: str) -> str:
    results = preference_retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results]) if results else "No preferences found."

def store_onam_recipe(recipe: str) -> str:
    if "onam" not in recipe.lower():
        return "Error: Only Onam-specific recipes can be stored."
    vector_store.add_texts([recipe])
    return "Onam recipe stored successfully."

def retrieve_onam_recipe(query: str) -> str:
    if "onam" not in query.lower():
        query = "Onam " + query
    results = retriever.get_relevant_documents(query)
    if results:
        combined_text = "\n".join([doc.page_content for doc in results])
        prompt = f"""
        Format the following text into coherent Onam recipes with ingredients and instructions. If ingredients are missing, infer them based on the text and common Onam dishes.
        Return in the format:
        Recipe: [Name]
        Ingredients: [Comma-separated list]
        Instructions: [Steps]
        ---
        If no recipe can be formed, return 'No Onam recipes found.'
        Text: {combined_text}
        """
        return llm.invoke(prompt).content.strip()
    return "No Onam recipes found."

# Updated Tools with priority
tools = [
    Tool(name="RecipeSearch", func=recipe_search, description="Primary: Search for non-Onam recipes using TheMealDB API."),
    Tool(name="ExtractIngredients", func=extract_ingredients, description="Extract ingredients from a recipe string."),
    Tool(name="NutritionAnalysis", func=nutrition_analysis, description="Primary: Analyze nutritional content of a recipe."),
    Tool(name="StorePreference", func=store_preference, description="Store user preferences (e.g., allergies) in the vector DB."),
    Tool(name="RetrievePreference", func=retrieve_preference, description="Retrieve user preferences from the vector DB."),
    Tool(name="PrivateOnamRecipeSearch", func=retrieve_onam_recipe, description="Search for Onam-specific recipes from private DB.")
]

# Updated System Prompt to force tool-first approach
system_prompt = """
You are a Chef Assistant specializing in recipes, nutrition, and dietary preferences.

PRIORITY RULES:
1. For any query about nutrition, calories, carbs, protein, fat, or similar — ALWAYS try the NutritionAnalysis tool first.
2. For recipe searches — ALWAYS try RecipeSearch or PrivateOnamRecipeSearch before attempting to rewrite or guess recipes. try the NutritionAnalysis tool for nutritional info if needed in the query of the recipie.
3. Only rewrite or modify recipes as a last resort if no results come from the tools.
4. Check user preferences before suggesting recipes and avoid conflicting ingredients.
5. For Onam recipes — store/retrieve using StoreOnamRecipe and PrivateOnamRecipeSearch.
6. Return multiple recipes separated by '---'. Provide clear ingredient lists and steps.
"""

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chef_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    agent_kwargs={"system_prompt": system_prompt}
)

def main():
    print("Welcome to the Chef Assistant!")
    while True:
        user_input = input("Enter your query (or 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        preferences = preference_retriever.get_relevant_documents("user preferences")
        preference_text = "\n".join([doc.page_content for doc in preferences]) if preferences else ""
        if preference_text:
            user_input += f" (Note: Consider user preferences - {preference_text})"
        response = chef_agent.run(user_input)
        print(response)

if __name__ == "__main__":
    main()


Welcome to the Chef Assistant!
Enter your query (or 'exit' to quit): Find me a recipie for Onam Payasam. low carb and fat


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: PrivateOnamRecipeSearch
Action Input: Onam Payasam, low carb, low fat
Observation: Recipe: Payasam
Ingredients: Cooked Payasam (e.g., rice/vermicelli cooked with milk/jaggery), Crushed Cardamom, Ghee, Raisins, Cashew nuts.
Instructions:
1. Ensure the Payasam base is cooked to your desired consistency.
2. Add crushed Cardamom and switch off from the stove.
3. In a separate pan, melt Ghee and fry Raisins and Cashew nuts until golden.
4. Pour the fried Raisins and Cashew nuts over the Payasam and serve.
---
Recipe: Coconut Milk Vegetable Curry
Ingredients: Coconut Milk, Sautéed vegetables (e.g., pumpkin, ash gourd, cowpeas), Green Chillies, Curry Leaves, Salt, Coconut Oil.
Instructions:
1. (Assumed: Prepare and sauté your chosen vegetables and other ingredients on high flame until par